In [2]:
import pandas as pd

dataset = pd.read_csv('dataset/gold_dataset_monthly.csv')

In [3]:
dataset.head()

,ano,mes,preco_ouro,ipc,indice_nyse
0,2007,1,631.06,202.42,9132.09
1,2007,2,666.44,203.50,9346.47
2,2007,3,654.19,205.35,9120.57
3,2007,4,679.64,206.69,9555.98
4,2007,5,667.59,207.95,9822.99


In [5]:
import statsmodels.api as sm

# Adiciona constante (intercepto)
X = sm.add_constant(dataset[['ipc', 'indice_nyse']])
y = dataset['preco_ouro']

# Ajuste do modelo
model = sm.OLS(y, X).fit()

# Resumo completo
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:             preco_ouro   R-squared:                       0.582
Model:                            OLS   Adj. R-squared:                  0.578
Method:                 Least Squares   F-statistic:                     131.8
Date:                sáb, 06 set 2025   Prob (F-statistic):           1.43e-36
Time:                        14:16:45   Log-Likelihood:                -1301.4
No. Observations:                 192   AIC:                             2609.
Df Residuals:                     189   BIC:                             2619.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
const       -2237.9571    258.867     -8.645      

In [70]:
from statsmodels.stats.diagnostic import het_breuschpagan

bp_test = het_breuschpagan(model.resid, model.model.exog)

# Resultados
labels = ['LM Stat', 'LM p-value', 'F-Stat', 'F p-value']
print(dict(zip(labels, bp_test)))

{'LM Stat': np.float64(2.5341104959041587), 'LM p-value': np.float64(0.2816598200925923), 'F-Stat': np.float64(1.223217984847338), 'F p-value': np.float64(0.3260227564348493)}


In [6]:
dataset['ipc'].corr(dataset['indice_nyse'])

np.float64(0.8851738797996995)

In [7]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

X_VIF = dataset[['ipc', 'indice_nyse']]
X_VIF = sm.add_constant(X_VIF)

vif_data = pd.DataFrame()
vif_data['Variavel'] = X_VIF.columns
vif_data['VIF'] = [variance_inflation_factor(X_VIF.values, i)
                   for i in range(X_VIF.shape[1])]

vif_data

,Variavel,VIF
0,const,280.201462
1,ipc,4.619637
2,indice_nyse,4.619637


In [12]:
import pandas as pd
import statsmodels.api as sm

# Criar defasagens (lags)
# Lag de 1 período no preço do ouro
dataset['preco_ouro_lag1'] = dataset['preco_ouro'].shift(12)

# Você pode adicionar lags das variáveis explicativas também, se quiser:
# dataset['ipc_lag1'] = dataset['ipc'].shift(1)
# dataset['indice_nyse_lag1'] = dataset['indice_nyse'].shift(1)

# Remover as linhas iniciais que ficaram com NaN por causa do shift
dataset_lagged = dataset.dropna()

# Definir X e y
X = dataset_lagged[['ipc', 'indice_nyse', 'preco_ouro_lag1']]
X = sm.add_constant(X)  # intercepto
y = dataset_lagged['preco_ouro']

# Ajustar o modelo OLS com lag
model = sm.OLS(y, X).fit()

# Resumo do modelo
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:             preco_ouro   R-squared:                       0.656
Model:                            OLS   Adj. R-squared:                  0.650
Method:                 Least Squares   F-statistic:                     112.0
Date:                sáb, 06 set 2025   Prob (F-statistic):           1.31e-40
Time:                        15:10:27   Log-Likelihood:                -1182.1
No. Observations:                 180   AIC:                             2372.
Df Residuals:                     176   BIC:                             2385.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
const            -607.6181    295.559     